# Event Viewer

## Imports

The first line sets the backend used by matplotlib. To be able to interact with the plots, make it say `%matplotlib nbagg`. For inline plots, use `%matplotlib inline`.

In [ ]:
%matplotlib inline

In [ ]:
import sys
sys.path.append('..')
import pytpc
from pytpc.constants import pi, degrees
import matplotlib.pyplot as plt
import matplotlib as mpl
from math import sin, cos
import numpy as np
import scipy.stats as ss
import seaborn as sns
import pandas as pd
from matplotlib.colors import LogNorm
from matplotlib import gridspec

In [ ]:
sns.set_style('darkgrid')

## TPC Parameters

Set these physical parameters to correct the 3D tracks for tilt and Lorentz angle, and to calibrate the *z* positions into distance units.

In [ ]:
tilt = 7*degrees
clock = 6.25  # MHz
emag = 15e3
bmag = 0.5691
ef = np.array([0, 0, emag])
bfield_orig = np.array([0, 0, bmag])
trans = np.array([[1, 0, 0],
                  [0, cos(-tilt), -sin(-tilt)],
                  [0, sin(-tilt), cos(-tilt)]])
bf = np.dot(trans, bfield_orig)
vd = pytpc.simulation.drift_velocity_vector(-3.32, emag, bmag, tilt)

## Loading Files

The event file should always be loaded.

In [ ]:
pads = pytpc.generate_pad_plane(rotation_angle=-108.*pytpc.pi/180.)
ef = pytpc.EventFile("/Users/josh/Documents/Data/Alphas-Dec14/run_0243_ps.evt")
#ef = evtdata.EventFile("/Users/josh/Documents/Data/Alphas-Dec14/OldEvts/run_0048.evt")

The files below should only be loaded if using event files without background subtraction.

In [ ]:
peds = evtdata.load_pedestals("/Users/josh/Dropbox/routing/Peds20141208-2.csv")
pm = evtdata.load_padmap("/Users/josh/Dropbox/routing/Lookup20141208.csv")

## Reading an Event

This one reads the event from the file:

In [ ]:
evt = ef[5]

If pedestals have not been subtracted already in the file, then run this cell:

In [ ]:
evt.traces['data'] -= [ numpy.ones(512,) * peds[tuple(addr)] for addr in evt.traces[['cobo','asad','aget','channel']] ]
evt.traces['data'] = ss.threshold(evt.traces['data'], threshmin=35)
evt.traces['pad'] = [ pm[tuple(addr)] for addr in evt.traces[['cobo','asad','aget','channel']] ]

## Plots

### Pad Plane

In [ ]:
p = pytpc.pad_plot(evt.hits(), pads)
# p.savefig('/Users/josh/Dropbox/padplane.pdf', transparent=True)

### 3D Plot

In [ ]:
cp = pytpc.chamber_plot(evt.xyzs())
cp.axes[0].view_init(azim=0, elev=10)
cp.savefig('/Users/josh/Dropbox/chamber.png', dpi=400, transparent=True)

In [ ]:
xyz = pd.DataFrame(evt.xyzs(peaks_only=False), columns=('x', 'y', 'z', 'act'))

In [ ]:
plt.figure(figsize=(7, 6))
gs = gridspec.GridSpec(2, 1, height_ratios=(1, 4), hspace=0.1)
with sns.axes_style('white'):
    plt.subplot(gs[0])
    plt.plot(range(512), evt.traces['data'].sum(0))
    plt.fill_between(range(512), evt.traces['data'].sum(0), alpha=0.7)
    plt.xlim(0, 511)
    plt.gca().invert_xaxis()
    plt.gca().get_xaxis().set_ticklabels([])
    plt.gca().get_yaxis().set_ticklabels([])
    plt.ylabel('Mesh')
    sns.despine()
    
with sns.axes_style('dark'):
    plt.subplot(gs[1])
    s = plt.scatter(xyz.z, xyz.y, marker='o', s=8, alpha=0.7, 
                    c=xyz.act, norm=LogNorm(), cmap=pytpc.tpcplot.ch_cm, linewidths=0)
    plt.xlim(0, 511)
    plt.ylim(-270, 270)
    plt.xlabel('Time [uncalibrated]')
    plt.ylabel('y [mm]')
    plt.gca().invert_xaxis()
    plt.savefig('/Users/josh/Dropbox/projection.pdf')

In [ ]:
sns.FacetGrid(xyz)

### Mesh Plot

In [ ]:
mesh = evt.traces['data'].sum(0)
plt.plot(mesh)
plt.xlabel('Time Bucket')
plt.ylabel('Activation')
plt.xlim(0, 512)

### Individual Channels

In [ ]:
lines = evt.traces['data']
for l in lines:
    plt.plot(l)
    plt.xlim(0, 512)
plt.xlabel('Time Buckets')
plt.ylabel('Activation [arb. units]')
plt.savefig('/Users/josh/Desktop/all_traces.png', dpi=200)

In [ ]:
sns.set_context('talk')

In [ ]:
sns.set_style('darkgrid')

In [ ]:
12.5e6 